In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-aa79t4y7
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-aa79t4y7
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=18c793f9a35f45d04db935713095f4b2b6a5d6c2634f8b97683bda04201619d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-2_gw6cud/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include<stdio.h>
#include<iostream>
#include<cuda.h>
using namespace std;
__global__ void arradd(int *x,int *y, int *z)
{
    int id=blockIdx.x * blockDim.x+threadIdx.x;
    z[id]=x[id]+y[id];
}
int main()
{
    int a[6]={1,2,3,4,7,9};
    int b[6]={1,2,3,4,9,7};
    int c[6];
    int *d,*e,*f;
    int i;
    cudaMalloc((void **)&d,6*sizeof(int));
    cudaMalloc((void **)&e,6*sizeof(int));
    cudaMalloc((void **)&f,6*sizeof(int));
 cudaMemcpy(d,a,6*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,6*sizeof(int),cudaMemcpyHostToDevice);
    arradd<<<2,3>>>(d,e,f);
 cudaMemcpy(c,f,6*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nSum of two arrays:\n ");
    for(i=0;i<6;i++)
    {
        cout<<c[i]<<" ";
    }
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    return 0;
}



Sum of two arrays:
 2 4 6 8 16 16 


In [ ]:
%%cu
#include<stdio.h>
#include<cuda.h>
#define row1 2 
#define col1 3 
#define row2 3 
#define col2 2


__global__ void matproduct(int *l,int *m, int *n)
{
    int x=blockIdx.x;
    int y=blockIdx.y;
    int k;
  
n[col2*y+x]=0;
for(k=0;k<col1;k++)
   {
    n[col2*y+x]=n[col2*y+x]+l[col1*y+k]*m[col2*k+x];
   }
}

int main()
{
    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int *d,*e,*f;
    int i,j;

    printf("\n Enter elements of first matrix of size 2*3\n");
    int k=1;
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col1;j++)
            {
                a[i][j]= k;
                k++;
            }
    }
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col1;j++)
        {
              printf("%d\t",a[i][j]);
        }
        printf("\n");
    }
 
    printf("\n Enter elements of second matrix of size 3*2\n");
        for(i=0;i<row2;i++)
        {
            for(j=0;j<col2;j++)
                {
                    b[i][j]=k;
                    k++;
                }
        }
        for(i=0;i<row2;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",b[i][j]);
        }
        printf("\n");
    }

    cudaMalloc((void **)&d,row1*col1*sizeof(int));
    cudaMalloc((void **)&e,row2*col2*sizeof(int));
    cudaMalloc((void **)&f,row1*col2*sizeof(int));

 cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,row2*col2*sizeof(int),cudaMemcpyHostToDevice);

dim3 grid(col2,row1);

    matproduct<<<grid,1>>>(d,e,f);

 cudaMemcpy(c,f,row1*col2*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nProduct of two matrices:\n ");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",c[i][j]);
        }
        printf("\n");
    }

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}


 Enter elements of first matrix of size 2*3
1	2	3	
4	5	6	

 Enter elements of second matrix of size 3*2
7	8	
9	10	
11	12	

Product of two matrices:
 58	64	
139	154	

